In [1]:
import pickle
import yaml
import numpy as np

In [14]:
env_types = [
    'environment.dynamic.dual_simple2arm_env.DualSimple2ArmEnv',
    'environment.dynamic.dual_kuka4_env.DualKuka4Env',
    'environment.dynamic.dual_kuka5_env.DualKuka5Env',
    'environment.dynamic.dual_kuka_env.DualKukaEnv',
    'environment.dynamic.triple_kuka_env.TripleKukaEnv',
    'environment.dynamic.triple_simple2arm_env.TripleSimple2ArmEnv',
]


bit_star_planner.ipynb         load_robot.ipynb
dataset.ipynb                  look_dataset.py
dataset_dynamic.ipynb          object_follow_trajectory.ipynb
grouping_robot.ipynb           robot_follow_trajectory.ipynb
load_environment.ipynb         rrt_star_planner.ipynb
load_object.ipynb              yaml_dataset


In [33]:
# with open('../local_dataset/arm2_1000_test_long_bidirectional.pkl', 'rb') as dataset_pkl:
#     dataset_pkl = pickle.load(dataset_pkl)

obs_files = [
    # '../local_dataset/arm2_env_1000_test_long_bidirectional.npz',
    '../local_dataset/arm2_env_1000_train_long_hard_bidirectional_part1.npz',
    '../local_dataset/arm2_env_1000_train_long_hard_bidirectional_part2.npz',
    '../local_dataset/arm2_env_1000_train_long_hard_bidirectional_part3.npz',
]

init_states_list = []
goal_states_list = []
obs_pos_list = []
obs_ori_list = []
obs_traj_list = []
for i in range(len(obs_files)):
    with np.load(obs_files[i]) as f:
        init_states_list.append(f['init_states'])
        goal_states_list.append(f['goal_states'])
        obs_pos_list.append(f['obs_pos'])
        obs_ori_list.append(f['obs_ori'])
        obs_traj_list.append(f['obs_traj'])
        # init_states_list = f['init_states']
        # goal_states_list = f['goal_states']
        # obs_pos_list = f['obs_pos']
        # obs_ori_list = f['obs_ori']
        # obs_traj_list = f['obs_traj']


init_states = np.concatenate(init_states_list)
goal_states = np.concatenate(goal_states_list)
obs_pos = np.concatenate(obs_pos_list)
obs_ori = np.concatenate(obs_ori_list)
obs_traj = np.concatenate(obs_traj_list)

print(len(init_states))

1183


In [34]:

env_type_id = 0
for id in range(min(len(obs_pos),1000)):
    obstacle = dict(
        type = 'objects.dynamic_object.DynamicObject',
        item = 'robot.simple2arm_robot.Simple2ArmRobot',
        trajectory = dict(
            type = 'objects.trajectory.WaypointLinearTrajectory',
            waypoints = obs_traj[id].tolist(),
        )
    )



    data = dict(
        env = dict(
            type=env_types[env_type_id],
            objects=[obstacle],
        ),
        start = init_states[id].tolist(),
        goal = goal_states[id].tolist(),
    )
    with open(f'./yaml_dataset/dual_simple2arm_env/hard/{id}.yml', 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)


# 3 simple

In [ ]:

obs_files = [
    # '../kuka_arm2_env_1000_test_long_bidirectional_01.npz',
    '../kuka_arm2_env_1000_long_bidirectional_hard_n2g_02.npz',
    '../kuka_arm2_env_1000_long_bidirectional_hard_n2g_03.npz',
    '../kuka_arm2_env_1000_long_bidirectional_hard_n2g_04.npz',
    '../kuka_arm2_env_1000_long_bidirectional_hard_n2g_05.npz',
]

graph_files = [
    # '../kuka_arm2_1000_test_long_bidirectional_01.pkl',
    '../kuka_arm2_1000_long_bidirectional_hard_n2g_02.pkl',
    '../kuka_arm2_1000_long_bidirectional_hard_n2g_03.pkl',
    '../kuka_arm2_1000_long_bidirectional_hard_n2g_04.pkl',
    '../kuka_arm2_1000_long_bidirectional_hard_n2g_05.pkl',

    ]


env_type_id = 3


graphs = []
init_states_list = []
goal_states_list = []
obs_pos_list = []
obs_ori_list = []
obs_traj_list = []

for file_idx in range(len(obs_files)):

    with np.load(obs_files[file_idx]) as f:
        init_states_list.append(f['init_states'])
        goal_states_list.append(f['goal_states'])
        obs_pos_list.append(f['obs_pos'])
        obs_ori_list.append(f['obs_ori'])
        obs_traj_list.append(f['obs_traj'])

    with open(graph_files[file_idx], 'rb') as f:
        load_graph = pickle.load(f)
        graphs.extend(load_graph)

init_states = np.concatenate(init_states_list)
goal_states = np.concatenate(goal_states_list)
obs_pos = np.concatenate(obs_pos_list)
obs_ori = np.concatenate(obs_ori_list)
obs_traj = np.concatenate(obs_traj_list)

for id in range(min(len(obs_pos),1000)):

    robot_obstacle = dict(
        type = 'objects.dynamic_object.DynamicObject',
        item='robot.kuka_robot.KukaRobot',
        trajectory = dict(
            type = 'objects.trajectory.WaypointLinearTrajectory',
            waypoints = obs_traj[id].tolist(),
        )
    )

    _, _, halfExtents_list, basePosition_list, _, _, _, _, _, _, _ = graphs[id]


    voxel_obstacles = list()
    for i in range(len(halfExtents_list)):
        voxel_obstacles.append(dict(
            type = 'objects.static.voxel.VoxelObject',
            half_extents = halfExtents_list[i].tolist(),
            base_position = basePosition_list[i].tolist(),
            base_orientation=[0, 0, 0, 1],
        ))

    all_obstacles = [robot_obstacle] + voxel_obstacles


    data = dict(
        env = dict(
            type=env_types[env_type_id],
            objects=all_obstacles,
        ),
        start = init_states[id].tolist(),
        goal = goal_states[id].tolist(),
    )


    with open(f'../hard/{id}.yml', 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)